In [0]:
from google.colab import files
uploaded = files.upload()

Saving buyers.csv to buyers.csv
Saving sellers.csv to sellers.csv


In [0]:
import itertools,pandas,random,io

def generate_individual(buyer,buyer_data,seller_data,seller):
    gene = []
    fitness = []
      
    for i in range(len(buyer)):
        gene.append(seller_data[random.randint(0,len(seller)-1)])
    individual = dict(zip(buyer_data,gene))
    return individual
    
def Population(population_size,buyer,buyer_data,seller_data,seller):
    population = []
    size = population_size
    for i in range(size):
        population.append(generate_individual(buyer,buyer_data,seller_data,seller))
    return population

def totalfitness(individual,buyer,seller,seller_inventory,buyer_csv_info,seller_price,seller_discount):
    
    buyer_remainingqty = dict(zip(buyer.id.tolist(),[0]*len(buyer))) 
    seller_soldamt =  dict(zip(seller.id.tolist(),[0]*len(seller))) 
    seller_inventory_temp = seller_inventory.copy()
    seller_original_inventory = seller_inventory.copy()
    buyer_csv_info = buyer_csv_info.copy()
    seller_price= seller_price.copy()
    totalfitness =0
    saving = 0
    valid_pair = []
    for key,value in individual.items():
        Ind_seller = value
        Ind_buyer = key               
        if seller_inventory_temp[Ind_seller]>= buyer_csv_info[Ind_buyer]:
            valid_pair.append(1)
            seller_soldamt[Ind_seller]+=buyer_csv_info[Ind_buyer]*seller_price[Ind_seller]
            seller_inventory_temp[Ind_seller] -= buyer_csv_info[Ind_buyer]
        else:
            valid_pair.append(0)
            buyer_remainingqty[Ind_buyer] = buyer_csv_info[Ind_buyer]-seller_inventory_temp[Ind_seller]
            seller_soldamt[Ind_seller]+=seller_inventory_temp[Ind_seller] * seller_price[Ind_seller]
            seller_inventory_temp[Ind_seller] = 0 
    
    if all(valid_pair):
        for key,value in seller_soldamt.items():
            saving += seller_soldamt[key] * (int(seller_discount[key].strip("%\n "))/100)
        
    return saving

def selTournament(population, k,individual_fitness):
    best = {}
    for i in range(k):
        ind = population[random.randint(0,len(population)-1)]
        fitness_best = 0
        fitness_ind = 0
        if not best:
            for i in individual_fitness:
                if best in i:
                    fitness_best = i[1]
        for i in individual_fitness:
            if ind in i:
                fitness_ind = i[1]
                    
        if not best or fitness_ind>fitness_best:
            best = ind
    return best

def crossover(p1,p2):
    rate = len(p1)
    ca = random.randint(0,rate)
    cb = random.randint(0,rate)
    
    if ca >cb:
        temp = cb
        cb = ca 
        ca = temp
    
    child1= []
    
   
    mom = list(p1.values())
    dad = list(p2.values())
    
    for i in range(rate):
        if i<ca :
            child1.append(dad[i])
            
        elif i<cb:
            child1.append(mom[i])
            
        else:
            child1.append(dad[i])
            
    return child1  

def changemutation(child,mutationRate,seller_data):
   
    rate = len(child)
    #choose the crossover point to change from the individual
    ca = random.randint(0,rate-1)
    #choose the seller to change
    selected_seller = seller_data[random.randint(0,len(seller_data)-1)]
    random_point = random.random()    
    if(random_point < mutationRate):
        child[ca] = selected_seller
    
    return child  

def bestfittest(individual_fitness):
    #search the best individual among population.
    best_fittest = individual_fitness[0]    
    for i in range(len(individual_fitness)):
        if best_fittest[1]<=individual_fitness[i][1]:
            best_fittest = individual_fitness[i]        
    return best_fittest[0]

def generate_poplation_fitness(population,buyer,seller,seller_inventory,buyer_csv_info,seller_price,seller_discount):
    population_fitness = []
    for i in range(len(population)):
        fitness = totalfitness(population[i],buyer,seller,seller_inventory,buyer_csv_info,seller_price,seller_discount)
        population_fitness.append([population[i],fitness])
    return population_fitness    

def main():
    buyer = pandas.read_csv(io.StringIO(uploaded['buyers.csv'].decode('utf-8')))
    buyer_csv_info = dict(zip(buyer.id.tolist(),buyer.quantity.tolist()))
    buyer_data = buyer.id.tolist()
    #reading from seller csv  
    seller = pandas.read_csv(io.StringIO(uploaded['sellers.csv'].decode('utf-8')))
    seller_csv_info = seller.to_dict()
    seller_inventory = dict(zip(seller.id.tolist(),seller.inventory.tolist()))
    seller_discount = dict(zip(seller.id.tolist(),seller.discount.tolist()))
    seller_price = dict(zip(seller.id.tolist(),seller.price.tolist()))
    seller_minamt = dict(zip(seller.id.tolist(),seller.minAmount.tolist()))
    seller_data = seller.id.tolist()
    population_size = 30
    termination = False#termination condition the GA will stop if the termination condition met
    k=5
    mutationRate = 0.01
    final_population = []
    generation = 0
    termination = 2000
    population = [] 
    population_fitness = []
    
    #initial population
    initial_population = Population(population_size,buyer,buyer_data,seller_data,seller)
    #initial population with fittness 
    individual_fitness = generate_poplation_fitness(initial_population,buyer,seller,seller_inventory,buyer_csv_info,seller_price,seller_discount)   
    
    print("initial population with fittness")
    for i in individual_fitness:
        print(i)
            
    population.append(initial_population)
    population_fitness.append(individual_fitness)
    while(generation<termination):
        new_population = []  
        new_population.append(bestfittest(population_fitness[generation]))
        for i in range(population_size-1):
            parent1 = selTournament(population[generation],k,population_fitness[generation])
            parent2 = selTournament(population[generation],k,population_fitness[generation])
            child = crossover(parent1,parent2)
            mutated_child = changemutation(child,mutationRate,seller_data)
            new_population.append(dict(zip(buyer_data,mutated_child))) 
        new_population_fittness = generate_poplation_fitness(new_population,buyer,seller,seller_inventory,buyer_csv_info,seller_price,seller_discount)
        generation+=1
        population.append(new_population)
        population_fitness.append(new_population_fittness)
    
    print("the latest population")
    
    for i in population_fitness[generation]:
        print(i)
    
    print("the best fittest is",bestfittest((population_fitness[generation])))
    
if __name__ == "__main__":
    main()    

initial population with fittness
[{'B-1': 'S-2', 'B-2': 'S-7', 'B-3': 'S-3', 'B-4': 'S-7', 'B-5': 'S-7', 'B-6': 'S-3', 'B-7': 'S-4', 'B-8': 'S-2', 'B-9': 'S-4', 'B-10': 'S-3', 'B-11': 'S-4', 'B-12': 'S-7', 'B-13': 'S-4'}, 0]
[{'B-1': 'S-7', 'B-2': 'S-5', 'B-3': 'S-2', 'B-4': 'S-3', 'B-5': 'S-3', 'B-6': 'S-6', 'B-7': 'S-7', 'B-8': 'S-7', 'B-9': 'S-3', 'B-10': 'S-7', 'B-11': 'S-4', 'B-12': 'S-4', 'B-13': 'S-6'}, 0]
[{'B-1': 'S-6', 'B-2': 'S-1', 'B-3': 'S-1', 'B-4': 'S-2', 'B-5': 'S-7', 'B-6': 'S-2', 'B-7': 'S-3', 'B-8': 'S-5', 'B-9': 'S-5', 'B-10': 'S-5', 'B-11': 'S-3', 'B-12': 'S-3', 'B-13': 'S-5'}, 0]
[{'B-1': 'S-5', 'B-2': 'S-5', 'B-3': 'S-2', 'B-4': 'S-2', 'B-5': 'S-3', 'B-6': 'S-7', 'B-7': 'S-1', 'B-8': 'S-5', 'B-9': 'S-5', 'B-10': 'S-1', 'B-11': 'S-5', 'B-12': 'S-2', 'B-13': 'S-1'}, 0]
[{'B-1': 'S-6', 'B-2': 'S-5', 'B-3': 'S-3', 'B-4': 'S-7', 'B-5': 'S-5', 'B-6': 'S-5', 'B-7': 'S-7', 'B-8': 'S-4', 'B-9': 'S-3', 'B-10': 'S-1', 'B-11': 'S-7', 'B-12': 'S-5', 'B-13': 'S-5'}, 0]
[{'B-1'